# **Voice Gender Recognition**

---



In [22]:
import pandas as pd

# Load the dataset
voice_data = pd.read_csv('/content/voice.csv')


print(voice_data.head())
print(voice_data.info())
print(voice_data.describe())

# Check for missing values
print(voice_data.isnull().sum())

   meanfreq        sd    median       Q25       Q75       IQR       skew  \
0  0.059781  0.064241  0.032027  0.015071  0.090193  0.075122  12.863462   
1  0.066009  0.067310  0.040229  0.019414  0.092666  0.073252  22.423285   
2  0.077316  0.083829  0.036718  0.008701  0.131908  0.123207  30.757155   
3  0.151228  0.072111  0.158011  0.096582  0.207955  0.111374   1.232831   
4  0.135120  0.079146  0.124656  0.078720  0.206045  0.127325   1.101174   

          kurt    sp.ent       sfm  ...  centroid   meanfun    minfun  \
0   274.402906  0.893369  0.491918  ...  0.059781  0.084279  0.015702   
1   634.613855  0.892193  0.513724  ...  0.066009  0.107937  0.015826   
2  1024.927705  0.846389  0.478905  ...  0.077316  0.098706  0.015656   
3     4.177296  0.963322  0.727232  ...  0.151228  0.088965  0.017798   
4     4.333713  0.971955  0.783568  ...  0.135120  0.106398  0.016931   

     maxfun   meandom    mindom    maxdom   dfrange   modindx  label  
0  0.275862  0.007812  0.007812  

In [23]:

!pip install librosa soundfile

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import librosa
from google.colab import files



voice_data = pd.read_csv('/content/voice.csv')

# Encode the labels (male = 0, female = 1)
le = LabelEncoder()
voice_data['label'] = le.fit_transform(voice_data['label'])


X = voice_data.drop('label', axis=1)
y = voice_data['label']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

def extract_features(audio_path, feature_names):
    y, sr = librosa.load(audio_path, sr=None)

    # Extract audio features
    pitch, _ = librosa.piptrack(y=y, sr=sr)
    mean_pitch = np.mean(pitch[pitch > 0]) if np.any(pitch > 0) else 0
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    rms = librosa.feature.rms(y=y)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)


    features = {
        'meanfreq': np.mean(spectral_centroid),
        'sd': np.std(spectral_centroid),
        'mean_pitch': mean_pitch,
        'rms': np.mean(rms),
        'spectral_bandwidth': np.mean(spectral_bandwidth)
    }

    # Ensure all features are present
    for missing_feature in feature_names:
        if missing_feature not in features:
            features[missing_feature] = 0

    # Return features in correct order
    return pd.DataFrame([features])[feature_names]


uploaded = files.upload()

# Extract features from the uploaded file
audio_path = list(uploaded.keys())[0]  # Get the name of the uploaded file
features_df = extract_features(audio_path, feature_names=X.columns)


features_df_scaled = scaler.transform(features_df)

# Make a prediction
prediction = svm_model.predict(features_df_scaled)

# Map the prediction back to the label
predicted_label = le.inverse_transform(prediction)[0]
print(f"Predicted Gender: {predicted_label}")


Accuracy: 0.97

Confusion Matrix:
[[437  15]
 [ 13 486]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       452
           1       0.97      0.97      0.97       499

    accuracy                           0.97       951
   macro avg       0.97      0.97      0.97       951
weighted avg       0.97      0.97      0.97       951



Saving male.mp3 to male (2).mp3
Predicted Gender: male
